# **RAG**
Retrieval Augmented Generation

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB**

### (1) Chroma DB 구성

* DB 경로 지정
    * 없으면, 새로 폴더를 만들며 DB 생성
    * 있으면, 기존 DB 연결

In [2]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./database",  # 경로 지정(현 위치에서 db 폴더 생성)
                    embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* DB 삭제는, 파이썬을 닫고 폴더를 삭제하면 됩니다.

### (2) INSERT

* 두가지 방법
    * (1) 단순 텍스트 입력 : 각 단위 텍스트를 리스트 형태로 입력 **.add_texts()**
    * (2) 텍스트와 메타데이터 입력 : 각 단위 텍스트와 메타정보를 함께 입력 **.add_documents()**

#### 1) .add_texts()

In [ ]:
input_list = ['test 데이터 입력1', 'test 데이터 입력2']

# 입력. 입력시 인덱스 저장(조회시 사용)
ind = database.add_texts(input_list)
ind

['8dd69f36-edc8-4939-bb02-b05f289a049b',
 '3dcb6d22-8905-46d1-b1cf-875977616a47']

In [ ]:
database.get(ind)

{'ids': ['3dcb6d22-8905-46d1-b1cf-875977616a47',
  '8dd69f36-edc8-4939-bb02-b05f289a049b'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['test 데이터 입력2', 'test 데이터 입력1'],
 'uris': None,
 'data': None}

이탤릭체 텍스트#### 2) .add_documents()

In [3]:
input_list2 = ['오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', '어제 주가는 큰 폭으로 상승했습니다.']
metadata = [{'category':'test'}, {'category':'test'}]

doc2 = [Document(page_content = input_list2[i], metadata = metadata[i]) for i in range(2)]
ind2 = database.add_documents(doc2)

In [4]:
database.get(ind2)

{'ids': ['4ecf6557-378f-4345-b66e-ce8eb309db61',
  '8cdb07b1-045e-4fb5-b043-fc092911e320'],
 'embeddings': None,
 'metadatas': [{'category': 'test'}, {'category': 'test'}],
 'documents': ['오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', '어제 주가는 큰 폭으로 상승했습니다.'],
 'uris': None,
 'data': None}

### (2) 조회

*이탤릭체 텍스트*#### 1) 전체 조회

In [ ]:
database.get()

{'ids': ['3dcb6d22-8905-46d1-b1cf-875977616a47',
  '8dd69f36-edc8-4939-bb02-b05f289a049b',
  'b3febe25-8b17-42e5-9ec2-fefc7f1ee20e',
  'd1418a17-377e-4377-9295-031fb0658674'],
 'embeddings': None,
 'metadatas': [None, None, {'category': 'test'}, {'category': 'test'}],
 'documents': ['test 데이터 입력2',
  'test 데이터 입력1',
  '어제 주가는 큰 폭으로 상승했습니다.',
  '오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.'],
 'uris': None,
 'data': None}

굵은 텍스트#### 2) 조건조회
* 일반적인 조건 조회를 지원하지 않음.
    * 그래서 전체를 다 불러와서
    * dataframe으로 변환 후 조건 조회

In [ ]:
data = database.get()
data = pd.DataFrame(data)
data

,ids,embeddings,metadatas,documents,uris,data
0,3dcb6d22-8905-46d1-b1cf-875977616a47,None,None,test 데이터 입력2,None,None
1,8dd69f36-edc8-4939-bb02-b05f289a049b,None,None,test 데이터 입력1,None,None
2,b3febe25-8b17-42e5-9ec2-fefc7f1ee20e,None,{'category': 'test'},어제 주가는 큰 폭으로 상승했습니다.,None,None
3,d1418a17-377e-4377-9295-031fb0658674,None,{'category': 'test'},오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.,None,None


In [ ]:
data.loc[data['metadatas'] == {'category': 'test'}]

,ids,embeddings,metadatas,documents,uris,data
2,b3febe25-8b17-42e5-9ec2-fefc7f1ee20e,None,{'category': 'test'},어제 주가는 큰 폭으로 상승했습니다.,None,None
3,d1418a17-377e-4377-9295-031fb0658674,None,{'category': 'test'},오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.,None,None


#### 3) 유사 문서 조회

In [ ]:
# 문서 조회1
query = "오늘 낮 기온은?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시

[Document(page_content='오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', metadata={'category': 'test'}), Document(page_content='어제 주가는 큰 폭으로 상승했습니다.', metadata={'category': 'test'}), Document(page_content='test 데이터 입력1')]
--------------------------------------------------
문서 내용: 오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.
문서 내용: 어제 주가는 큰 폭으로 상승했습니다.
문서 내용: test 데이터 입력1


In [ ]:
# 문서 조회2 : 유사도 점수도 함께 조회
query = "오늘 낮 기온은?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', metadata={'category': 'test'}), 0.2255800180603788), (Document(page_content='어제 주가는 큰 폭으로 상승했습니다.', metadata={'category': 'test'}), 0.3811239242860163), (Document(page_content='test 데이터 입력1'), 0.49518008096659605)]
--------------------------------------------------
유사도 점수 : 0.22558, 문서 내용: 오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.
유사도 점수 : 0.38112, 문서 내용: 어제 주가는 큰 폭으로 상승했습니다.
유사도 점수 : 0.49518, 문서 내용: test 데이터 입력1


**굵은 텍스트**### (3) 삭제

* 문서의 id 값으로 삭제합니다.

In [ ]:
data

,ids,embeddings,metadatas,documents,uris,data
0,3dcb6d22-8905-46d1-b1cf-875977616a47,None,None,test 데이터 입력2,None,None
1,8dd69f36-edc8-4939-bb02-b05f289a049b,None,None,test 데이터 입력1,None,None
2,b3febe25-8b17-42e5-9ec2-fefc7f1ee20e,None,{'category': 'test'},어제 주가는 큰 폭으로 상승했습니다.,None,None
3,d1418a17-377e-4377-9295-031fb0658674,None,{'category': 'test'},오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.,None,None


In [ ]:
ids = ['3dcb6d22-8905-46d1-b1cf-875977616a47', '8dd69f36-edc8-4939-bb02-b05f289a049b']
database.delete(ids = ids)

In [ ]:
database.get()

{'ids': ['b3febe25-8b17-42e5-9ec2-fefc7f1ee20e',
  'd1418a17-377e-4377-9295-031fb0658674'],
 'embeddings': None,
 'metadatas': [{'category': 'test'}, {'category': 'test'}],
 'documents': ['어제 주가는 큰 폭으로 상승했습니다.', '오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.'],
 'uris': None,
 'data': None}

## **3.데이터 벡터화**

### (1) DF to Vector DB

* 샘플데이터 : 오픈소스 생성형 AI에서 주의해야 할 10가지 사항
    * 기사를 csv로 만든 것입니다.

원문보기: https://www.ciokorea.com/news/337152#csidxc7d1d11066fad86a15937e4c3b29c6d



In [ ]:
data = pd.read_csv('sample.csv', encoding='utf-8')
data

,구분,내용
0,낯선 라이선스 약관,현재 오픈소스 AI 라이선스 유형은 다양한 만큼 매우 복잡하다. 오픈소스 AI 모델...
1,낯선 라이선스 약관,메타와 유사하게 애플은 ‘애플 샘플 코드 라이선스’하에 오픈ELM을 출시했다. 이 ...
2,전문성 부족,오픈소스는 스스로 해야 하는 작업이 많을 수 있다. 기업은 코드를 다운로드할 수는 ...
3,전문성 부족,업계 전체에 전문성이 부족하다는 점은 새로운 문제를 야기한다. 원래 오픈소스의 장점...
4,탈옥,LLM 공격 중 유명한 것에 탈옥(jailbreak)이라는 수법이 있다. 탈옥은 의...
5,탈옥,반면에 악의적인 공격자는 접근하기 쉬운 오픈소스 모델을 무료로 다운로드하여 자신의 ...
6,학습 데이터가 가진 위험,"예술가, 작가, 기타 저작권 소유자들이 대형 AI 기업을 상대로 소송을 제기하고 있..."
7,학습 데이터가 가진 위험,노탈의 수기스는 “대형 벤더는 학습 데이터를 구입하고 소송을 벌이는 데 쓸 돈이 있...
8,새로운 보안 위협,생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출...
9,새로운 보안 위협,또 다른 공격 벡터는 모델의 시스템 프롬프트이다. 수기스에 따르면시스템 프롬프트는 ...


In [ ]:
# Chroma 데이터베이스 인스턴스 생성
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)

In [ ]:
# 데이터프레임의 텍스트 열(시리즈)을 리스트로 변환
text_list = data['내용'].tolist()

# 리스트 내용을 각각 document로 변환
documents = [Document(page_content=text) for text in text_list]

print(documents)

In [ ]:
# Insert
database.add_documents(documents)

['10a0acad-177f-46d0-ba51-1dd3e1eaa237',
 'da91293d-e6a1-43af-aaa4-61e4084836bf',
 '3d8b388e-8063-4982-bf70-a60a4f09a736',
 'c8724250-8e83-44b3-8505-b69610522720',
 'f67961a4-b27e-4a9d-b44d-c7251dbbf5f0',
 '2eb36f7c-23dd-46d7-92d9-277ab5775f5a',
 'e9ef9f6a-cc89-496d-a7d8-ddc9783ae21d',
 'e41e988c-e9be-4772-ac8d-8f73939359ca',
 '82478d04-bd6a-4f63-b036-8ba73967e705',
 'beb1203d-cae6-4497-8460-9c7f3f509e6a',
 '6b443556-df5a-4afd-8bff-1fec9a0baf13',
 'e7ca8446-17d2-4e50-a707-0d6239f18c4a',
 'b1fcb519-697e-497c-a049-2c0122955a9c']

In [ ]:
database.get()

{'ids': ['10a0acad-177f-46d0-ba51-1dd3e1eaa237',
  '2eb36f7c-23dd-46d7-92d9-277ab5775f5a',
  '3d8b388e-8063-4982-bf70-a60a4f09a736',
  '6b443556-df5a-4afd-8bff-1fec9a0baf13',
  '82478d04-bd6a-4f63-b036-8ba73967e705',
  'b1fcb519-697e-497c-a049-2c0122955a9c',
  'beb1203d-cae6-4497-8460-9c7f3f509e6a',
  'c8724250-8e83-44b3-8505-b69610522720',
  'da91293d-e6a1-43af-aaa4-61e4084836bf',
  'e41e988c-e9be-4772-ac8d-8f73939359ca',
  'e7ca8446-17d2-4e50-a707-0d6239f18c4a',
  'e9ef9f6a-cc89-496d-a7d8-ddc9783ae21d',
  'f67961a4-b27e-4a9d-b44d-c7251dbbf5f0'],
 'embeddings': None,
 'metadatas': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'documents': ['현재 오픈소스 AI 라이선스 유형은 다양한 만큼 매우 복잡하다. 오픈소스 AI 모델을 이용하려면 상업적으로 사용해도 괜찮은지, 수정 및 배포가 가능한지, 사내 코드 베이스에 안전하게 통합할 수 있는지 등을 알아봐야 한다. 여기에 몇 가지 새로운 문제가 등장했다. 일단 이전에는 볼 수 없는 제약이 오픈소스 라이선스에 적용됐다.\n메타의 라마(Llama) 라이선스를 예로 보자. 라마는 높은 인기를 자랑하는 오픈소스 LLM이다. 메타는 라마에 대해 ‘개방형 접근권을 제공하고 잠재적인 오용 문제를 해결하기 맞춤형 상용 라이

### (2) 질문으로 유사도 높은 문서 조회하기

In [ ]:
# 문서 조회 : 유사도 점수도 함께 조회
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"   # 질문할 문장
k = 5                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출될 수 있는 영역이 더 많다. LLM은 여러 방면에서 악의적인 공격을 받을 수 있다. 수기스에 따르면, 이들은 관리가 제대로 이루어지지 않는 프로젝트의 개발팀에 침투하여 소프트웨어에 바로 악성 코드를 추가할 수 있다. 거기서 끝나지 않고 훈련 데이터, 미세 조정 또는 가중치까지 오염시킬 수 있다.\n수기스는 “해커는 예시 악성 코드로 모델을 재학습시켜 사용자 인프라에 침입할 수 있다”라며 “또는 가짜 뉴스와 잘못된 정보로 모델을 훈련시킬 수도 있다”라고 설명했다.'), 0.2518438275766568), (Document(page_content='사실 많은 사람이 AI 표준에 대해 이야기할 때 윤리, 개인정보 보호, 설명 가능성 등을 이야기한다. 실제로 작년 12월에 발표된 ISO/IEC 42001 표준과 같은 작업이 해당 영역을 다루고 있다. 그리고 지난 4월 미국 국립표준기술연구소(NIST)는 AI 관련 공통 언어를 제시하는 등 AI 표준 계획 초안을 발표했다. 여기에선 주로 위험과 거버넌스 문제에 초점을 맞추고 있다. 그럼에도 기술 표준에 관해서는 아직 정해진 것이 많지 않다.\n클라우드 네이티브 컴퓨팅 재단의 CIO인 테일러 돌잘은 “표준에 대한 논의는 매우 초기 단계에 머물러 있다”라며 ”단 데이터 분류, 학습 데이터, API, 프롬프트에 대한 표준 형식에 대해 의미 있는 논의가 이뤄지고 있다”라고 말했다. 그래도 아직 ‘논의’하는 수준일 뿐이다.'), 0.264080306752224), (Document(page_content='오픈소스 그룹 일각에서는 AI 모델에 가드레일(허용 가능한 범위를 두는 일종의 가이드라인 또는 도구)을 두는 것 자체를 반대하기도 한다. 모델에 아무런 제한 없어야 더 나은 성능을 발휘할 것이라 믿는 곳도 있다. 기업은 현재 사용하는 오픈소스 모델이 가드레일에 대해 어떤 방향을 추구하는지조차 잘 모

### (3) 질문에 대한 답변 받기
* 절차
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 질문과 유사도높은 문서로 프롬프트 구성
    * GPT에 질문하고 답변 받기

* RetrievalQA
    * 랭체인에서 제공하는 체인 함수
    * RAG + QA

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
retriever = database.as_retriever()
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)

print(result["result"])

생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 프로젝트 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나, 훈련 데이터, 미세 조정, 가중치를 오염시키는 공격이 있을 수 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시키는 위협도 있을 수 있습니다. 따라서 보안을 강화하고 데이터의 안전성을 보장하는 것이 중요합니다.


* 유사도 높은 문서 4개를 조회하는 것이 기본값임.

In [ ]:
result

{'query': '생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?',
 'result': '생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 프로젝트 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나, 훈련 데이터, 미세 조정, 가중치를 오염시키는 공격이 있을 수 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시키는 위협도 있을 수 있습니다. 따라서 보안을 강화하고 데이터의 안전성을 보장하는 것이 중요합니다.',
 'source_documents': [Document(page_content='생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출될 수 있는 영역이 더 많다. LLM은 여러 방면에서 악의적인 공격을 받을 수 있다. 수기스에 따르면, 이들은 관리가 제대로 이루어지지 않는 프로젝트의 개발팀에 침투하여 소프트웨어에 바로 악성 코드를 추가할 수 있다. 거기서 끝나지 않고 훈련 데이터, 미세 조정 또는 가중치까지 오염시킬 수 있다.\n수기스는 “해커는 예시 악성 코드로 모델을 재학습시켜 사용자 인프라에 침입할 수 있다”라며 “또는 가짜 뉴스와 잘못된 정보로 모델을 훈련시킬 수도 있다”라고 설명했다.'),
  Document(page_content='사실 많은 사람이 AI 표준에 대해 이야기할 때 윤리, 개인정보 보호, 설명 가능성 등을 이야기한다. 실제로 작년 12월에 발표된 ISO/IEC 42001 표준과 같은 작업이 해당 영역을 다루고 있다. 그리고 지난 4월 미국 국립표준기술연구소(NIST)는 AI 관련 공통 언어를 제시하는 등 AI 표준 계획 초안을 발표했다. 여기에선 주로 위험과 거버넌스 문제에 초점을 맞추고 있다. 그럼에도 기술 표준에 관해서는 아직 정해진 것이 많지 않다.\n클라우드 네이티브 컴퓨팅 재단의 CIO인 테일러 돌잘은 “표준에 대한 논의는 매우 초기 단계에 머물러 있다”라며 ”단 데이터 분류, 학습 데이터

* 유사도 높은 문서 k개로부터 프롬프트 구성하여 질의하기

In [ ]:
k = 5
retriever = database.as_retriever(search_kwargs={"k": k})

qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)
result

{'query': '생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?',
 'result': '생성형 AI 도입 시 예상되는 보안 위협은 다양합니다. 일부 위협은 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터를 오염시키는 것일 수 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시키는 등의 위협이 있을 수 있습니다. 이러한 위협으로부터 보호하기 위해서는 모델을 테스트하고 평가하며, 내부에 보호 장치를 마련하는 등의 조치가 필요할 수 있습니다.',
 'source_documents': [Document(page_content='생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출될 수 있는 영역이 더 많다. LLM은 여러 방면에서 악의적인 공격을 받을 수 있다. 수기스에 따르면, 이들은 관리가 제대로 이루어지지 않는 프로젝트의 개발팀에 침투하여 소프트웨어에 바로 악성 코드를 추가할 수 있다. 거기서 끝나지 않고 훈련 데이터, 미세 조정 또는 가중치까지 오염시킬 수 있다.\n수기스는 “해커는 예시 악성 코드로 모델을 재학습시켜 사용자 인프라에 침입할 수 있다”라며 “또는 가짜 뉴스와 잘못된 정보로 모델을 훈련시킬 수도 있다”라고 설명했다.'),
  Document(page_content='사실 많은 사람이 AI 표준에 대해 이야기할 때 윤리, 개인정보 보호, 설명 가능성 등을 이야기한다. 실제로 작년 12월에 발표된 ISO/IEC 42001 표준과 같은 작업이 해당 영역을 다루고 있다. 그리고 지난 4월 미국 국립표준기술연구소(NIST)는 AI 관련 공통 언어를 제시하는 등 AI 표준 계획 초안을 발표했다. 여기에선 주로 위험과 거버넌스 문제에 초점을 맞추고 있다. 그럼에도 기술 표준에 관해서는 아직 정해진 것이 많지 않다.\n클라우드 네이티브 컴퓨팅 재단의 CIO인 테일러 돌잘은 “표준에 대한 논의는 매우 초기 단계에 머물러 있다”라며 ”단 데

## **4.관리자용 DB 생성 및 관리**